In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from sklearn.preprocessing import OrdinalEncoder


warnings.filterwarnings('ignore')

## 데이터 전처리 과정

In [2]:
path = 'data/'
train = pd.read_csv('{}train.csv'.format(path))
test = pd.read_csv('{}test.csv'.format(path))

In [3]:
#쓸모없는 칼럼 삭제
for df in (train,test):
    df.drop(['FLAG_MOBIL'], axis=1, inplace=True)   
    df.drop(columns=['index'], inplace=True)
    
del df

In [4]:
# 1st

def year_month_total(df_1, df_2, column, column_name):
    """[나이, 고용연수 파생변수를 생성하기 위한 함수입니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        column ([생성할 컬럼의 기준 컬럼 명]): [생성할 컬럼의 기준 컬럼 명을 입력해주세요]
        column_name ([생성할 컬럼의 이름]): [연단위, 연+월단위로 표시된 파생변수 이름입니다.]
    """
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']

    if (column == 'DAYS_BIRTH') == True:    
        for dataset,name in zip(df_list,names):
            dataset['{}_year'.format(column_name)] = dataset[column].abs()//365
            dataset['{}_month'.format(column_name)] = (dataset[column].abs()%365)//30
            dataset['{}_total'.format(column_name)] = dataset['{}_year'.format(column_name)] + round(dataset['{}_month'.format(column_name)] * (1/12), 2)
            dataset.drop(columns=['{}_month'.format(column_name)], inplace=True)
        
            print('{}set에 {}_year, {}_total 컬럼을 생성하였습니다. \n'.format(name,column_name,column_name))
    
    else:
        for dataset,name in zip(df_list,names):
            print('{}set에 근무일자가 잘못기입된 {} cases에 대한 처리를 시작합니다.'.format(name, len(dataset[dataset['DAYS_EMPLOYED']>=0])))
            dataset['DAYS_EMPLOYED'].sort_values().value_counts()
            dataset['DAYS_EMPLOYED'] = dataset['DAYS_EMPLOYED'].replace(365243, 0)
            dataset.query('DAYS_EMPLOYED < 0')['DAYS_EMPLOYED'].sort_values()

            dataset['{}_year'.format(column_name)] = dataset[column].abs()//365
            dataset['{}_month'.format(column_name)] = (dataset[column].abs()%365)//30
            dataset['{}_total'.format(column_name)] = dataset['{}_year'.format(column_name)] + round(dataset['{}_month'.format(column_name)] * (1/12), 2)
            dataset.drop(columns=['{}_month'.format(column_name)], inplace=True)
        
            print('{}set에 {}_year, {}_total 컬럼을 생성하였습니다. \n'.format(name, column_name,column_name))
        
    del df_list,names
    

In [5]:
year_month_total(train, test,'DAYS_BIRTH','age')

trainset에 age_year, age_total 컬럼을 생성하였습니다. 

testset에 age_year, age_total 컬럼을 생성하였습니다. 



In [6]:
year_month_total(train, test,'DAYS_EMPLOYED','work')

trainset에 근무일자가 잘못기입된 4438 cases에 대한 처리를 시작합니다.
trainset에 work_year, work_total 컬럼을 생성하였습니다. 

testset에 근무일자가 잘못기입된 1697 cases에 대한 처리를 시작합니다.
testset에 work_year, work_total 컬럼을 생성하였습니다. 



In [7]:
# 2nd

def occyp_type_nan(df_1, df_2):
    """[직업 컬럼 내의 Nan 값을 처리합니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
    """
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']

    for dataset,name in zip(df_list,names):
        dataset['occyp_type'] = dataset['occyp_type'].fillna('Nan')
        print('직업 컬럼의 Nan value {}명 중 {}명이 실제 고용일수가 0일입니다. 따라서 jobless로 처리합니다.'.format(len(dataset.loc[dataset['occyp_type'] == 'Nan']), len(dataset.loc[(dataset['occyp_type'] == 'Nan') & (dataset['DAYS_EMPLOYED'] == 0)])))
        dataset.loc[(dataset['occyp_type'] == 'Nan') & (dataset['DAYS_EMPLOYED'] == 0), 'occyp_type'] = 'jobless'
        print('나머지 {}명 데이터는 no_data로 처리합니다.'.format(len(dataset.loc[dataset['occyp_type'] == 'Nan'])))
        dataset.loc[dataset['occyp_type'] == 'Nan', 'occyp_type'] = 'no data'
        print('{}set의 직업 Nan value를 처리하였습니다.\n'.format(name))
        
    del df_list, names

occyp_type_nan(train, test)

직업 컬럼의 Nan value 8171명 중 4438명이 실제 고용일수가 0일입니다. 따라서 jobless로 처리합니다.
나머지 3733명 데이터는 no_data로 처리합니다.
trainset의 직업 Nan value를 처리하였습니다.

직업 컬럼의 Nan value 3152명 중 1697명이 실제 고용일수가 0일입니다. 따라서 jobless로 처리합니다.
나머지 1455명 데이터는 no_data로 처리합니다.
testset의 직업 Nan value를 처리하였습니다.



### 1. ID컬럼 생성

In [66]:
# 개인정보를 나열한 칼럼 ID 추가 (만약 여기서 credit도 추가하게 되면 고유 ID가 12099명으로 증가함)
# 12099명으로 증가하는 경우는 유추하자면 credit 정보가 최신화되지 않았기 때문에 발생한다.
# 따라서 가장 최근의 credit을 user_list로 반영하기 위한 작업을 실시한다.
# 과거와 최근을 구별하는 방법은 bigin_month(카드 생성 후 기간)을 기준으로 시행한다.


def ID_col(df_1, df_2):
    """[데이터 셋 전처리 함수입니다.]

    Args:
        df_1 ([Dataset]): [train 데이터셋을 입력해주세요]
        df_2 ([Dataset]): [test 데이터셋을 입력해주세요]
    """
        
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    a = []
    change_credit = 0
    names = ['train', 'test']
    
    
    for name, dataset, in zip(names, df_list):
        dataset['ID'] = dataset['gender'].astype(str) +'.'+ dataset['car'].astype(str)+'.' + dataset['reality'].astype(str)+'.' + \
                dataset['child_num'].astype(str)+'.' + dataset['income_total'].astype(str) +'.'+ dataset['income_type'].astype(str) +'.'+ \
                dataset['edu_type'].astype(str)+'.' + dataset['family_type'].astype(str)+'.' + dataset['house_type'].astype(str) +'.'+ \
                dataset['DAYS_BIRTH'].astype(str) +'.'+ dataset['DAYS_EMPLOYED'].astype(str) +'.'+ dataset['work_phone'].astype(str) +'.'+ \
                dataset['phone'].astype(str) +'.'+ dataset['email'].astype(str) +'.' + dataset['family_size'].astype(str)+'.' + dataset['occyp_type'].astype(str)
        
        dataset = dataset.sort_values(['ID','begin_month'], ascending=[True, False])
        dataset.reset_index(inplace=True)
        dataset.drop(columns={'index'}, inplace=True)

        print('해당 Data 내의 고유 ID 수는 {} 입니다. OridonalEncoder를 이용힌 변환을 시작합니다.'.format(dataset['ID'].nunique()))
        encode = OrdinalEncoder()
        dataset[['ID']] = encode.fit_transform(dataset[['ID']])

        # Nth_card 컬럼 추가 (ID가 같은데 카드 개수가 여러 개인 사람을 대상으로 몇 번째 카드를 만들었는지 생성해준다)
        print('Column(Nth_card)을 생성합니다.')
        for i in tqdm(range(dataset['ID'].nunique())):
            Nth_card = len(dataset[dataset['ID']==i]) + 1
            
            while Nth_card > 0:
                Nth_card = Nth_card -1
                a.append(Nth_card)
                if Nth_card == 1:
                    break
                
        dataset['Nth_card'] = a        
        a.clear()
        print('신규로 카드를 개설한 사람의 수 : {}'.format(len(dataset[dataset['begin_month']==0.0]['ID'].unique())))
        
        
        if (dataset['ID'].nunique() > 8000)==True:
            print('credit 정보를 확인하고 시간이 지나며 credit이 악화된 사례가 있는지 확인합니다.')
            for i in tqdm(range(dataset['ID'].nunique())):
                num = dataset.loc[dataset['ID'] == i]['credit'].to_list()
                for j in range(len(num)-1):
                    if (num[j] <= num[j+1]) == True:
                        pass
                    else :
                        change_credit += 1
                        break
            print('{}명(전체 중 {:.2f}%)은 credit이 개선되지 않고 악화된 사례가 존재합니다.'.format(change_credit, 
                                                                        (change_credit/dataset['ID'].nunique())*100))  
        else:
            print('credit 정보가 없는 testset입니다.')
            
        print('dataset(user_list_{})을 저장합니다. \n'.format(name))    
        dataset.to_csv('{}user_list_{}.csv'.format(path, name))
        
    del names, a

In [67]:
ID_col(train, test)

해당 Data 내의 고유 ID 수는 8759 입니다. OridonalEncoder를 이용힌 변환을 시작합니다.
Column(Nth_card)을 생성합니다.


100%|██████████| 8759/8759 [00:05<00:00, 1698.42it/s]


신규로 카드를 개설한 사람의 수 : 214
credit 정보를 확인하고 시간이 지나며 credit이 악화된 사례가 있는지 확인합니다.


100%|██████████| 8759/8759 [00:07<00:00, 1196.86it/s]


1842명(전체 중 21.03%)은 credit이 개선되지 않고 악화된 사례가 존재합니다.
dataset(user_list_train)을 저장합니다. 

해당 Data 내의 고유 ID 수는 5585 입니다. OridonalEncoder를 이용힌 변환을 시작합니다.
Column(Nth_card)을 생성합니다.


100%|██████████| 5585/5585 [00:04<00:00, 1312.55it/s]


신규로 카드를 개설한 사람의 수 : 82
credit 정보가 없는 testset입니다.
dataset(user_list_test)을 저장합니다. 



In [19]:
train = pd.read_csv('{}user_list_train.csv'.format(path))
test = pd.read_csv('{}user_list_test.csv'.format(path))

In [59]:
change_credit = 0

for i in tqdm(range(dataset['ID'].nunique())):
    num = dataset.loc[dataset['ID'] == i]['credit'].to_list()
    for j in range(len(num)-1):
        if (num[j] <= num[j+1]) == True:
            pass
        else :
            change_credit += 1
            break
    
print('{}명(전체 중 {:.2f}%)은 credit이 개선되지 않고 악화된 사례가 존재합니다.'.format(change_credit, 
                                                                      (change_credit/dataset['ID'].nunique())*100))
        

100%|██████████| 8759/8759 [00:05<00:00, 1593.96it/s]


1842명(전체 중 21.03%)은 credit이 개선되지 않고 악화된 사례가 존재합니다.


### faimly_size 조정

In [247]:
def replace_value_family(df_1, df_2, num_of_outlier):
    """[Family_size의 outlier를 대치해주는 작업을 시행합니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        num_of_outlier ([outlier_standard]): [family_size의 outlier 기준을 입력하세요]
    """
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']

    for dataset,name in zip(df_list,names):
        print('family_size와 child_num의 다중공선성 문제로 child_num 컬럼을 삭제해줍니다.')
        dataset.drop(columns=['child_num'], inplace=True)
        print('family_size {}명 이상인 사람의 수 : {}'.format(num_of_outlier, 
                                                        len(dataset.loc[dataset['family_size'] >= num_of_outlier])))
        sub = dataset.loc[dataset['family_size']>= num_of_outlier]
        for i in tqdm(range(len(sub))):
            a = int(dataset[dataset['family_type'] == sub['family_type'].values[i]].mean()['family_size'])
            dataset.loc[(dataset['family_size']>= num_of_outlier), 'family_size'] = a
        
        print('family_size {}명 이상인 사람의 수 : {}'.format(num_of_outlier, 
                                                        len(dataset.loc[dataset['family_size'] >= num_of_outlier])))        # # family_size가 6을 초과하는 사람을 다른 값으로 대치하는 작업

        # family_size를 조정하고 난 후에 1인당 소득으로 total_income을 scale_down 해줌.
        print('1인당 소득으로 소득 수준을 조정합니다.')
        dataset['income_mean'] = dataset['income_total'] / dataset['family_size']
        print('{}데이터 셋 처리 완료.\n'.format(name))
    
    del sub,df_list, names
        
replace_value_family(train, test, 7)

family_size와 child_num의 다중공선성 문제로 child_num 컬럼을 삭제해줍니다.
family_size 7명 이상인 사람의 수 : 15


100%|██████████| 15/15 [00:41<00:00,  2.77s/it]


family_size 7명 이상인 사람의 수 : 0
1인당 소득으로 소득 수준을 조정합니다.
train데이터 셋 처리 완료.

family_size와 child_num의 다중공선성 문제로 child_num 컬럼을 삭제해줍니다.
family_size 7명 이상인 사람의 수 : 10


100%|██████████| 10/10 [00:03<00:00,  2.73it/s]

family_size 7명 이상인 사람의 수 : 0
1인당 소득으로 소득 수준을 조정합니다.
test데이터 셋 처리 완료.



In [42]:
names = ['train', 'test']
path = 'data/'

for name,dataset in zip(names, [user_list_train, user_list_test]):
    dataset.drop(columns=['DAYS_BIRTH','DAYS_EMPLOYED','level_0','index'], inplace=True)
    dataset.to_csv('{}final_set_{}.csv'.format(path,name))